In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


## Preprocesado de datos

##### En este apartado, en primer lugar cargaremos las primersa líbrerias y tambíen  los datos.

##### El segundo paso es visualizar los datos y gestionar los nulos de la mejor forma posible.

**TIP :** *Para que tu codigo sea más eficaz y en casa de necesitarlo realizar futuras actualizaciones es recomendable el uso de funciones*

In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test= pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

Visualizamos como es el dataframe

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
def preprocesado(df):
    df=df.drop(['Alley'], axis=1) # Eliminamos la columna Alley
    df=df.dropna(subset=['MasVnrType']) # Eliminamos las filas con nulos
    df['BsmtFinType2']=df['BsmtFinType2'].fillna('Unf') # Asumimos los nulos como Unf (Sin terminar)
    df['LotFrontage']=df['LotFrontage'].fillna(df['LotFrontage'].median()) # Imputamos los nulos con la mediana
    df['BsmtQual']=df['BsmtQual'].fillna(0) # Imputamos los nulos por 0 (No tienen sotano)
    df['BsmtCond']=df['BsmtCond'].fillna(0) # Imputamos los nulos por 0 (No tienen sotano)
    df['BsmtExposure']=df['BsmtExposure'].fillna(0) # Imputamos los nulos por 0 (No tienen sotano)
    df['BsmtFinType1']=df['BsmtFinType1'].fillna(0) # Imputamos los nulos por 0 (No tienen sotano)
    df=df.dropna(subset=['Electrical']) # Eliminamos el error en Electrical
    df['FireplaceQu']=df['FireplaceQu'].fillna(0) # Imputamos los nulos por 0 (No tienen chimenea)
    df['GarageType']=df['FireplaceQu'].fillna('NonHave') # Imputamos los nulos por 'NonHave' (No tienen garaje)
    df['GarageYrBlt']=df['GarageYrBlt'].fillna(0) # Imputamos los nulos por 0 (Indicamos que los que no tienen garaje)
    df['GarageFinish']=df['GarageFinish'].fillna('NonHave') # Imputamos los nulos por 'NonHave' (No tienen garaje)
    df['GarageQual']=df['GarageQual'].fillna(0) # Indicamos que el garaje no existe
    df['GarageCond']=df['GarageCond'].fillna(0) # Indicamos que el garaje no existe
    df=df.drop(['PoolQC'], axis=1) # Eliminamos la columna Calidad de la piscina (Praticamente toda vacia)
    df['MiscFeature']=df['MiscFeature'].fillna('NonHave') # Indicamos que no tiene características extras
    df['Fence']=df['Fence'].fillna('NonHave') # Indicamos que no tiene vallas
    return df

In [6]:
train=preprocesado(train)

## Features Selection

##### En esta sección vamos a seleccionar solo las variables que creamos que tienen sentido para nuestro modelo, cada elección tiene que tener un porque.
##### Sobrecargar un modelo puede provocar largos periodos de espera y hacer que nuestro modelo no encuentre las mejores relaciones.

*Es necesario leerse el apartado Data Field de la página '[Datos](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data)' de la descripción de la competición*

## Técnicas multiples de regresión 

### Regresión lineal multiple